In [2]:
from tensorflow.keras.models import Sequential, load_model
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from tqdm import tqdm  # 导入 tqdm

# 读取训练好的模型
model = load_model('predict_model_lstm.h5')

# 读取需要预测的数据
Import_csv_file = './merged_output.csv'
Import_df = pd.read_csv(Import_csv_file)



# 获取表头
x_headers = Import_df.columns.tolist()

# 选择输入列（控制和动力相关特征）
X = Import_df[['Time','AntriebsmomentVA', 'AntriebsmomentHA', 'Lenkwinkel', 'BremsmomentVA', 'BremsmomentHA']].values

# 归一化输入数据
scaler_X = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)

# 获取输出数据
Exoport_csv_file = './output.csv'
Exoport_merged_data = pd.read_csv(Exoport_csv_file)

# 获取输出列头
y_headers = Exoport_merged_data.columns.tolist()

# 选择输出列（如车辆状态和行为相关特征）
y = Exoport_merged_data[['KS_X', 'KS_Y', 'Gierwinkel', 'Geschw_X', 'Geschw_Y', 'Giergeschwindigkeit']].values

# 归一化输出数据
scaler_y = MinMaxScaler(feature_range=(0, 1))
y_scaled = scaler_y.fit_transform(y)

# 将数据转换为新的输入形式，包含历史时间步
def create_dataset_with_history(X, y, history_steps=50):
    X_data = []
    for i in range(history_steps, len(X)):
        # 过去 history_steps 步的输入
        past_x = X[i - history_steps:i]

        # 过去 history_steps 步的输出
        past_y = y[i - history_steps:i]

        # 拼接过去的输入和输出
        combined_input = np.hstack((past_x, past_y))
        X_data.append(combined_input)

    return np.array(X_data)

# 参数配置
history_steps = 50

# 创建输入数据
X_data = create_dataset_with_history(X_scaled, y_scaled, history_steps)

# 预测时需要的输入格式
X_data_reshaped = X_data.reshape(X_data.shape[0], history_steps, -1)

# 批量预测优化
batch_size = 64
predictions = []

for i in tqdm(range(0, len(X_data_reshaped), batch_size), desc="批量预测进度"):
    # 提取批量数据
    batch_seq = X_data_reshaped[i:i + batch_size]

    # 批量预测
    batch_pred = model.predict(batch_seq, verbose=0)
    predictions.append(batch_pred)

# 将所有批量预测结果拼接
predictions = np.vstack(predictions)

# 反归一化预测结果
predictions_rescaled = scaler_y.inverse_transform(predictions)

# 修正预测结果的列名问题
# 将列名调整为正确的顺序并匹配数据长度
y_headers_corrected = y_headers[-predictions_rescaled.shape[1]:]  # 从末尾取正确数量的列名
pred_df = pd.DataFrame(predictions_rescaled, columns=y_headers_corrected)

# 合并预测结果与原始输入数据
merged_data = pd.concat([Import_df.iloc[history_steps:].reset_index(drop=True), pred_df], axis=1)

# 保存结果到 CSV 文件
merged_data.to_csv('pred_result_modified.csv', index=False)


批量预测进度: 100%|██████████████████████████████████████████████████████████████████| 390/390 [00:21<00:00, 17.92it/s]
